In [7]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
df = sns.load_dataset("titanic")

In [9]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [11]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [13]:
# filling null values

from sklearn.impute import SimpleImputer

Imp = SimpleImputer( strategy = "median" )
df[["age"]] = Imp.fit_transform(df[["age"]])

imp = SimpleImputer(strategy = "most_frequent")
df[["embarked"]] = imp.fit_transform(df[["embarked"]])
df[["deck"]] = imp.fit_transform(df[["deck"]])
df[["embark_town"]] = imp.fit_transform(df[["embark_town"]])

In [15]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [20]:
# encoding
cols = ["sex" , "embarked" , "class" , "who" , "adult_male" , "deck" , "embark_town" , "alive" , "alone"]
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])

In [21]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,1,2,2,0,0
1,1,1,0,38.0,1,0,71.2833,0,0,2,0,2,0,1,0
2,1,3,0,26.0,0,0,7.9250,2,2,2,0,2,2,1,1
3,1,1,0,35.0,1,0,53.1000,2,0,2,0,2,2,1,0
4,0,3,1,35.0,0,0,8.0500,2,2,1,1,2,2,0,1


In [22]:
#train-test-split
X = df.drop(["survived"] , axis = 1)
y = df["survived"]

X_train , X_test , y_train , y_test = train_test_split(
    X , y , test_size = 0.3 , random_state = 42
)

In [33]:
# model-training & evaluation
from sklearn.metrics import f1_score , accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [39]:
param_grid = {
    "svc__C": [0.1, 1, 10],
    "svc__kernel": ["linear", "rbf"],
    "svc__gamma": ["scale", "auto"]
}


pipeline = Pipeline([
    ("scalar" , StandardScaler()),
    ("svc" , SVC())
])

CV = GridSearchCV(
    pipeline,
    param_grid,
    cv = 5
)

CV.fit(X_train , y_train)
y_pred = CV.predict(X_test)

In [41]:
CV.best_params_

{'svc__C': 0.1, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}

In [42]:
svc = SVC(C = 0.1 , gamma = "scale" , kernel = "linear")
svc.fit(X_train , y_train)
y_pred = svc.predict(X_test)

print("Accuracy :" , accuracy_score(y_test , y_pred))
print("F1-score :" , f1_score(y_test , y_pred))

Accuracy : 1.0
F1-score : 1.0
